### Установка библиотек

In [ ]:
!pip install python-telegram-bot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install python-telegram-bot --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# перезапускаем среду выполнения

In [ ]:
!pip install pymorphy2
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
import datetime

### Блок обработки текста

In [ ]:
morpher = MorphAnalyzer()
stop_words = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [ ]:
# удаляем слово "время" из множества stop_words
stop_words.discard('время')

In [ ]:
def preprocess_txt(input_txt):
    txt = "".join(i for i in input_txt.strip() if i not in exclude).split()
    txt = [morpher.parse(i.lower())[0].normal_form for i in txt]
    txt = [i for i in txt if i not in stop_words and i != ""]
    return txt

In [ ]:
def is_intent_time(input_txt):
  res = False
  if len(input_txt) < 6:
    for word in input_txt:
      if word == 'время' or word == 'час':
        res = True

  return res

In [ ]:
def is_intent_5k(input_txt):
  res = False
  if len(input_txt) < 6:
    for word in input_txt:
      if word == 'пятёрочка' or word == 'пятерочка':
        res = True

  return res  

In [ ]:
var_otv = {'yes': 1, 'no' : 2, 'dialog' : 0}

In [ ]:
def is_yes_no(input_txt):
  var = var_otv.get('dialog')
  if input_txt == 'да' or input_txt == 'Да':
    var = var_otv.get('yes')
  elif input_txt == 'нет' or  input_txt =='Нет':
    var =  var_otv.get('no')
  elif (len(input_txt.split()) > 1) and (input_txt.split()[0]=='да' or input_txt.split()[0] == 'Да'):
    var = var_otv.get('yes')

  return var

### Блок парсинга Пятёрочки

In [ ]:
from pathlib import Path
import time
import json
import requests

In [ ]:
class Parse5ka:
    headers = {"User-Agent": "ucheba"}

    def __init__(self, start_url: str):
        self.start_url = start_url

    def _get_response(self, url, *args, **kwargs) -> requests.Response:
        while True:
            response = requests.get(url, *args, **kwargs, headers=self.headers)
            if response.status_code in (200, 301, 304):
                return response
            time.sleep(1)

    def run(self):
        prod = dict()
        for product in self._parse(self.start_url):
            prod[product['name']] = product['current_prices'].get('price_promo__min')
        return prod
        
    def _parse(self, url):
        while url:
            response = self._get_response(url)
            data: dict = response.json()
            url = data.get("next")
            for product in data.get("results", []):
                yield product

In [ ]:
parser = Parse5ka("https://5ka.ru/api/v2/special_offers/")
product = parser.run()

In [ ]:
def price5k(prod = 'колбаса'):
  return [f'{key} - {val}р.' for key, val in product.items() if key.lower().find(prod.lower())!=-1]

### Блок соединения с телеграм ботом

In [ ]:
import os
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [ ]:
# Создадим соединение с ботом
updater = Updater("5393420496:AAFk_-R2VQH94tQUsaDqjYZC6twdwMltzPA", use_context=True)  # Токен API к Telegram
dispatcher = updater.dispatcher

In [ ]:
def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Добрый день! Я учебный бот @Aleksey_educ_bot')
    update.message.reply_text('Могу просто поболтать')
    update.message.reply_text('Сказать рецепт блюда')
    update.message.reply_text('Показать скидки на продукты в Пятёрочке')
    update.message.reply_text('Подсказать наше время')

In [ ]:
intent = []
last_txt = ''

In [ ]:
def textMessage(update: Update, context: CallbackContext):
  otv_var = is_yes_no(update.message.text)
  global last_txt, price5k
  
  if otv_var == var_otv.get('yes'):
    if intent[0] == 'time':
      intent.clear()
      today = datetime.datetime.now()
      update.message.reply_text(f'у нас {today.strftime("%H.%M")}') 
    elif intent[0] == '5k':
      intent.clear()

      if len(update.message.text.split()) > 1:
        prod = price5k(" ".join(update.message.text.split()[1:]))
        if len(prod) > 0:
          for mes in prod:
            update.message.reply_text(mes)
        else:
            update.message.reply_text('Извините, на указанный продукт нет скидок')
      else:
        # колбаса
        for mes in price5k():
          update.message.reply_text(mes)
  
  elif otv_var == var_otv.get('no'):
    intent.clear()
    input_txt = last_txt
    last_txt = ''
  elif otv_var == var_otv.get('dialog'):
    input_txt = preprocess_txt(update.message.text)

  if otv_var == var_otv.get('dialog') and len(intent) == 1:  
    update.message.reply_text('выберете вариант ответа "да" или "нет"')
  elif otv_var == var_otv.get('dialog') and is_intent_time(input_txt):
    update.message.reply_text('вы хотите узнать который час?')
    intent.append('time')
    last_txt = input_txt
  elif otv_var == var_otv.get('dialog') and is_intent_5k(input_txt):
    update.message.reply_text('вы хотите узнать скидки в Пятёрочке? (после Да, можно указать продукт)')
    intent.append('5k')
    last_txt = input_txt
  elif otv_var == var_otv.get('dialog') or otv_var == var_otv.get('no'): 
    update.message.reply_text('test')

In [ ]:
# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", startCommand))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage))

# Start the Bot
updater.start_polling()
updater.idle()